In [20]:
from datetime import datetime, timedelta

def parse_date(date_str):
    """Parse date string to datetime object."""
    return datetime.strptime(date_str, "%d-%m-%Y %H:%M")

def handle_missing_values(line_num, start_time_str, end_time_str):
    """Handle lines with missing Time or Time Out values."""
    if not start_time_str or not end_time_str:
        print(f"Warning in line {line_num}: Missing Time or Time Out value. Skipping line.")
        return None, None
    return parse_date(start_time_str), parse_date(end_time_str)

def analyze_file(file_path):
    """Analyze employee data from the given file."""
    # Dictionary to store employee information
    employees = {}

    with open(file_path, 'r', encoding='latin-1') as file:
        header = next(file)  # Skip the header line
        for line_num, line in enumerate(file, start=2):
            # Assuming each line in the file has the format: "Position ID,Position Status,Time,Time Out,..."
            values = line.strip().split(',')

            # Check if the line has the expected number of values
            if len(values) < 9:
                print(f"Error in line {line_num}: Insufficient number of values. Skipping line.")
                continue

            name = values[-2]
            position = values[1]  # Assuming Position is in the second column

            # Extracting Time and Time Out columns
            start_time_str = values[2]
            end_time_str = values[3]

            # Handle missing values
            start_time, end_time = handle_missing_values(line_num, start_time_str, end_time_str)
            if start_time is None or end_time is None:
                continue

            # Update employee information
            if name not in employees:
                employees[name] = {'position': position, 'shifts': []}
            employees[name]['shifts'].append({'start_time': start_time, 'end_time': end_time})

    for name, data in employees.items():
        shifts = data['shifts']

        # Check for 7 consecutive days of work
        consecutive_days = any((shifts[i + 1]['start_time'] - shifts[i]['end_time']).days == 1 for i in range(len(shifts) - 1))

        # Check for less than 10 hours but greater than 1 hour between shifts
        short_breaks = any((shifts[i + 1]['start_time'] - shifts[i]['end_time']).seconds < 36000 and
                           (shifts[i + 1]['start_time'] - shifts[i]['end_time']).seconds > 3600
                           for i in range(len(shifts) - 1))

        # Check for more than 14 hours in a single shift
        long_shifts = any((shift['end_time'] - shift['start_time']).seconds > 50400 for shift in shifts)

        # Print results
        if consecutive_days:
            print(f"{name} has worked for 7 consecutive days.")
        if short_breaks:
            print(f"{name} has less than 10 hours but greater than 1 hour between shifts.")
        if long_shifts:
            print(f"{name} has worked for more than 14 hours in a single shift.")

if __name__ == "__main__":
    file_path = "/content/cleaned_data.csv"
    analyze_file(file_path)


Warning in line 1472: Missing Time or Time Out value. Skipping line.
 XAis" has worked for 7 consecutive days.
 XAis" has less than 10 hours but greater than 1 hour between shifts.
 KeWWeMh" has worked for 7 consecutive days.
 CeghaW" has less than 10 hours but greater than 1 hour between shifts.
 FeXipe" has worked for 7 consecutive days.
 JAsMiW" has worked for 7 consecutive days.
 XeEWeX" has worked for 7 consecutive days.
 PedrE" has worked for 7 consecutive days.
 PedrE" has less than 10 hours but greater than 1 hour between shifts.
 XaCar" has less than 10 hours but greater than 1 hour between shifts.
 CasiaWE" has worked for 7 consecutive days.
 DaA" has worked for 7 consecutive days.
 IgWaciE" has less than 10 hours but greater than 1 hour between shifts.
 WiXXiaC Ha" has less than 10 hours but greater than 1 hour between shifts.
 DaWieX" has worked for 7 consecutive days.
 AXfredE" has worked for 7 consecutive days.
 JEhW" has worked for 7 consecutive days.
 CaWpreeM" has work

In [12]:
import pandas as pd

# Assuming 'df' is your DataFrame
df = pd.read_csv('/content/Assignment_Timecard.xlsx - Sheet1.csv')

# Convert 'Time', 'Time Out', and 'Timecard Hours (as Time)' columns to datetime format
df['Time'] = pd.to_datetime(df['Time'], errors='coerce')
df['Time Out'] = pd.to_datetime(df['Time Out'], errors='coerce')
df['Timecard Hours (as Time)'] = pd.to_datetime(df['Timecard Hours (as Time)'], format='%H:%M:%S', errors='coerce').dt.time

# Convert 'Pay Cycle Start Date' and 'Pay Cycle End Date' columns to datetime format
df['Pay Cycle Start Date'] = pd.to_datetime(df['Pay Cycle Start Date'], errors='coerce')
df['Pay Cycle End Date'] = pd.to_datetime(df['Pay Cycle End Date'], errors='coerce')

# Drop rows with missing values in key columns
df.dropna(subset=['Position ID', 'Employee Name', 'File Number'], inplace=True)

# Save the cleaned data to a new CSV file or update the existing one
df.to_csv('cleaned_data.csv', index=False)
